In [1]:
from pprint import pprint

print('Hello, World!')

Hello, World!


In [2]:
from datetime import datetime
import requests

import ta
import pandas as pd

from lumibot.strategies import Strategy
from lumibot.backtesting import PandasDataBacktesting
from lumibot.entities import Asset, Bars, Data

base_asset = 'ETH'
quote_asset = 'USDC'
resolution = '5'  # 5 minute

class HydraStrategy(Strategy):
    parameters = {
        'fast_ema_period': 5,
        'slow_ema_period': 20,
        "rsi_period": 14,
        "rsi_overbought": 70,
        "rsi_oversold": 30,
    }

    def initialize(self):
        self.set_market('24/7')

    def on_trading_iteration(self):
        self.base = Asset(base_asset, Asset.AssetType.CRYPTO)
        self.quote = Asset(quote_asset, Asset.AssetType.CRYPTO)
        # print(f'Portfolio value: {self.portfolio_value}')

        # If cash exceeds initial budget by more than 10%, log a warning that target has been reached
        if self.cash >= self.initial_budget + (self.initial_budget * 0.1):
            print(f"Porfolio value {self.cash} exceeds initial budget {self.initial_budget} by more than 10%.")
            self.close_positions([self.base])
            raise Exception("Target reached, stopping strategy.")
        elif self.cash <= self.initial_budget - (self.initial_budget * 0.5):
            print(f"Porfolio value {self.cash} below initial budget {self.initial_budget} by more than 10%.")
            self.close_positions([self.base])
            raise Exception("Stop loss reached, stopping strategy.")
        else:
            bars: Bars = self.get_historical_prices(
                asset=self.base,
                quote=self.quote,
                timestep='5 minutes',
                length=100,
                timeshift=100 if self.first_iteration else 0
            )
            # print(bars.df)
            # raise Exception("Stopping after one iteration for demonstration purposes.")
            fast_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["fast_ema_period"])
            slow_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["slow_ema_period"])
            rsi = ta.momentum.RSIIndicator(bars.df['close'], self.parameters["rsi_period"])

            if (
                (fast_ema.ema_indicator().iloc[-1] > slow_ema.ema_indicator().iloc[-1])
                and
                (self.parameters["rsi_oversold"] < rsi.rsi().iloc[-1] < self.parameters["rsi_overbought"])
                ):
                # print("Golden cross detected!")
                if not self.get_position(self.base):
                    # print("Placing buy order!")
                    order = self.create_order(
                        asset=self.base,
                        quote=self.quote,
                        quantity=100,
                        side='buy'
                    )
                    self.submit_order(order)
                    if not self.is_backtesting: self.wait_for_order_execution(order)
            elif (
                (fast_ema.ema_indicator().iloc[-1] < slow_ema.ema_indicator().iloc[-1])
                and
                (self.parameters["rsi_oversold"] < rsi.rsi().iloc[-1] < self.parameters["rsi_overbought"])
                ):
                # print("Death cross detected!")
                if self.get_position(self.base):
                    # print("Placing sell order!")
                    order = self.create_order(
                        asset=self.base,
                        quote=self.quote,
                        quantity=100,
                        side='sell'
                    )
                    self.submit_order(order)
                    if not self.is_backtesting: self.wait_for_order_execution(order)


strategy = HydraStrategy()

backtesting_start = datetime(2025, 8, 23)
backtesting_end = datetime(2025, 8, 30)

url = "https://api.orderly.org/v1/tv/history"
querystring = {
    "symbol": f"PERP_{base_asset}_{quote_asset}",
    "resolution": resolution,
    "from": str(int(backtesting_start.timestamp())),
    "to": str(int(backtesting_end.timestamp())),
}
response = requests.get(url, params=querystring)
data = response.json()
df = pd.DataFrame({
    'datetime': data.get('t', []),
    'open': data.get('o', []),
    'high': data.get('h', []),
    'low': data.get('l', []),
    'close': data.get('c', []),
    'volume': data.get('v', [])
})
df['datetime'] = pd.to_datetime(df['datetime'], unit='s')
df.set_index('datetime', inplace=True)
asset = Asset(base_asset, Asset.AssetType.CRYPTO)
quote = Asset(quote_asset, Asset.AssetType.CRYPTO)
pandas_data = {
    asset: Data(
        asset,
        df,
        timestep='minute',
        quote=quote,
    )
}
# print(df.tail())

strategy.backtest(
    PandasDataBacktesting,
    backtesting_start=pandas_data[asset].datetime_start,
    backtesting_end=pandas_data[asset].datetime_end,
    budget=2000000,
    pandas_data=pandas_data,
    benchmark_asset=asset,
    quote_asset=quote,
)

Lumibot version 4.0.3
2025-08-30 09:36:47,587 | INFO | .env file loaded from: /home/balo/projects/hydra-strategy/.env
2025-08-30 09:36:47,591 | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2025-08-30 09:36:50,665 | INFO | [HydraStrategy] Starting backtest...
2025-08-30 09:36:50,667 | INFO | Backtesting starting...
Progress || 100.00%  [Elapsed: 0:05:20 ETA: 0:00:00] Portfolio Val: 2,090,808.00

/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


[8208:8208:0100/000000.331269:ERROR:content/zygote/zygote_linux.cc:660] write: Broken pipe (32)


{'cagr': np.float64(14.317919052370318),
 'volatility': np.float64(0.11200983189219099),
 'sharpe': np.float64(127.46639119947808),
 'max_drawdown': {'drawdown': np.float64(0.004179440150543209),
  'date': Timestamp('2025-08-26 13:50:00-0400', tz='America/New_York')},
 'romad': np.float64(3425.798321459249),
 'total_return': np.float64(0.04585000000000239)}

In [3]:
print(datetime(2025, 8, 26).timestamp())

1756162800.0


/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.
